In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch03SparkToolSet").getOrCreate()

In [3]:
sc = spark.sparkContext
sc._jsc.sc().uiWebUrl().get()

'http://172.16.1.109:4046'

In [4]:
for x in sc._conf.getAll():
    if 'PROXY' in x[0]:
        print(x[1])
print(spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES'))
print(spark.conf.get('spark.driver.appUIAddress'))

rm01.itversity.com
http://rm01.itversity.com:19288/proxy/application_1528589352821_34767
http://rm01.itversity.com:19288/proxy/application_1528589352821_34767
http://172.16.1.109:4046


In [5]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/spark-guide/data/retail-data/by-day/*.csv")

In [6]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [7]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [8]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [9]:
staticDataFrame.show()

+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|         InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:...|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:...|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:...|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:...|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:...|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:...|     0.85|   14075.0|United Kingdom|
|   580538|    2312

In [15]:
sc.version

'2.0.0.2.5.0.0-1245'

In [11]:
# COMMAND ----------
from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   13199.0|[2011-10-19 20:00...| 76.32000000000001|
|   16170.0|[2011-10-19 20:00...|324.39000000000004|
|   14261.0|[2011-10-19 20:00...|             219.7|
|   17434.0|[2011-10-19 20:00...|            327.93|
|   13755.0|[2011-10-19 20:00...|            297.85|
+----------+--------------------+------------------+
only showing top 5 rows



In [12]:
# COMMAND ----------
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/user/kranthidr/spark-guide/data/retail-data/by-day/*.csv")

In [13]:
# COMMAND ----------
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [14]:
# COMMAND ----------
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [18]:
# COMMAND ----------

spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(20)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|      null|[2010-12-20 19:00...|31347.479999999938|
|   18102.0|[2010-12-06 19:00...|          25920.37|
|      null|[2010-12-09 19:00...|25399.560000000012|
|      null|[2010-12-16 19:00...|25375.189999999766|
|      null|[2010-12-05 19:00...|23395.099999999904|
|      null|[2010-12-02 19:00...| 23021.99999999999|
|   15749.0|[2011-01-10 19:00...|           22998.4|
|   17450.0|[2011-01-10 19:00...|18620.199999999993|
|   14156.0|[2011-01-13 19:00...|16774.719999999998|
|      null|[2010-12-13 19:00...|15929.879999999974|
|      null|[2010-12-08 19:00...|15354.279999999955|
|      null|[2011-01-16 19:00...|12735.669999999953|
|      null|[2010-11-30 19:00...|12584.299999999988|
|   14646.0|[2011-01-13 19:00...|10389.060000000003|
|   15061.0|[2010-12-01 19:00...| 9407.339999999998|
|      null|[2011-01-09 19:00...| 9207.2400000

In [19]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `window` DESC
  """)\
  .show(20)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   13684.0|[2011-02-26 19:00...| 64.23000000000002|
|   12748.0|[2011-02-26 19:00...|275.24999999999983|
|   14147.0|[2011-02-26 19:00...|             182.8|
|   13630.0|[2011-02-26 19:00...|299.14000000000004|
|   16745.0|[2011-02-26 19:00...|            486.65|
|   16407.0|[2011-02-26 19:00...|202.09999999999997|
|   16598.0|[2011-02-26 19:00...|            145.05|
|   16493.0|[2011-02-26 19:00...|             240.0|
|   13767.0|[2011-02-26 19:00...|317.86999999999995|
|   14745.0|[2011-02-26 19:00...|             240.6|
|   15738.0|[2011-02-26 19:00...|            333.92|
|   13610.0|[2011-02-26 19:00...|239.17000000000004|
|   17516.0|[2011-02-26 19:00...| 327.8300000000001|
|   14810.0|[2011-02-26 19:00...|180.65000000000003|
|   15708.0|[2011-02-26 19:00...| 644.7500000000002|
|   15547.0|[2011-02-26 19:00...| 421.68999999

In [17]:
purchaseByCustomerPerHour.writeStream\
    .format("console")\
    .queryName("customer_purchases_2")\
    .outputMode("complete")\
    .start()

In [16]:
# COMMAND ----------

from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)

In [17]:
preppedDataFrame.show()

+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|         InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+-----------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:...|     1.79|   14075.0|United Kingdom|     Monday|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:...|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:...|     1.65|   14075.0|United Kingdom|     Monday|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:...|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:...|     2.55|   14075.0|United Kingdom|     Monday|
|   580538|    21544|SKULLS  WAT

In [18]:
# COMMAND ----------

trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")

In [19]:
# COMMAND ----------
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")

In [20]:
# COMMAND ----------

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")

In [21]:
# COMMAND ----------

from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

In [22]:
# COMMAND ----------

from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])

In [23]:
# COMMAND ----------

fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [24]:
# COMMAND ----------

transformedTraining = fittedPipeline.transform(trainDataFrame)

In [31]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [32]:
# COMMAND ----------

from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
  .setK(20)\
  .setSeed(1)

In [33]:
# COMMAND ----------

kmModel = kmeans.fit(transformedTraining)

In [34]:
# COMMAND ----------

transformedTest = fittedPipeline.transform(testDataFrame)

In [35]:
transformedTest.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_index: double (nullable = true)
 |-- day_of_week_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [36]:
kmModel.computeCost(transformedTest)

539019994.0519797

In [37]:
kmModel.computeCost(transformedTraining)

76546274.08334887

In [61]:
from pyspark.sql import Row
sc.parallelize([Row(1), Row(2), Row(3)]).toDF().show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 81.0 failed 4 times, most recent failure: Lost task 0.3 in stage 81.0 (TID 963, wn04.itversity.com): java.io.IOException: Cannot run program "/home/kranthidr/.virtualenvs/pyspark-lab/bin/python3.5": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:163)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:89)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:65)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:114)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:128)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.R

In [64]:
spark.range(1,4).toDF("number").show()

+------+
|number|
+------+
|     1|
|     2|
|     3|
+------+

